In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import talib as ta

pd.options.display.float_format = '{:.2f}'.format


In [118]:
def calcula_stop_atr(df, close_col, atr_col, desvio, periodo):
    """
    Calcula os valores de Stop ATR (Average True Range) para um DataFrame de dados financeiros.

    :param df: DataFrame contendo os dados.
    :param close_col: Nome da coluna do DataFrame contendo os valores de fechamento.
    :param atr_col: Nome da coluna do DataFrame contendo os valores de ATR (Average True Range).
    :param desvio: Valor do desvio a ser usado no cálculo do Stop ATR.
    :param periodo: Período a ser considerado no cálculo do Stop ATR.
    :return: DataFrame enriquecido com as colunas 'Stop ATR C' (para condições de compra) e 'Stop ATR V' (para condições de venda).
    """
    df = df.copy()
    
    # Calcula a diferença entre os valores de fechamento e o produto de ATR e desvio
    df['ATR_abs C'] = abs(close_col - (atr_col * desvio))
    df['Stop ATR_aux'] = np.NaN

    # Calcula os valores do Stop ATR C (para condições de compra)
    for i in range(len(df)):
        idx = df.index[i]
        if i >= periodo:
            if close_col.iloc[i] >= df['Stop ATR_aux'].iloc[i - 1]:
                if df['ATR_abs C'].iloc[i] > df['Stop ATR_aux'].iloc[i - 1]:
                    df.at[idx, 'Stop ATR_aux'] = df['ATR_abs C'].iloc[i]
                else:
                    df.at[idx, 'Stop ATR_aux'] = df['Stop ATR_aux'].iloc[i - 1]
            else:
                df.at[idx, 'Stop ATR_aux'] = df['ATR_abs C'].iloc[i]

    # Armazena o valor calculado do Stop ATR C e remove colunas intermediárias
    df['Stop ATR C'] = df['Stop ATR_aux'].shift(1)
    df = df.drop(['Stop ATR_aux', 'ATR_abs C'], axis=1)
    
    # Calcula a diferença entre os valores de fechamento e o produto de ATR e desvio
    df['ATR_abs V'] = abs(close_col + (atr_col * desvio))
    df['Stop ATR_aux'] = np.NaN

    # Calcula os valores do Stop ATR V (para condições de venda)
    for i in range(len(df)):
        idx = df.index[i]
        if i >= periodo:
            if close_col.iloc[i] < df['Stop ATR_aux'].iloc[i - 1]:
                if df['ATR_abs V'].iloc[i] < df['Stop ATR_aux'].iloc[i - 1]:
                    df.at[idx, 'Stop ATR_aux'] = df['ATR_abs V'].iloc[i]
                else:
                    df.at[idx, 'Stop ATR_aux'] = df['Stop ATR_aux'].iloc[i - 1]
            else:
                df.at[idx, 'Stop ATR_aux'] = df['ATR_abs V'].iloc[i]

    # Armazena o valor calculado do Stop ATR V e remove colunas intermediárias
    df['Stop ATR V'] = df['Stop ATR_aux'].shift(1)
    df = df.drop(['Stop ATR_aux', 'ATR_abs V'], axis=1)
    
    return df

In [25]:
def calculate_satrc_satrv(data, periodo, desvio):
    """
    Calcula os valores de SATRc e SATRv.
    """
    satrc = [0]
    satrv = [0]

    # Calcular o ATR e adicioná-lo ao DataFrame
    #data.loc[:, 'ATR'] = ta.SMA(data['TrueRange'], periodo)

    #data.loc[:, 'ATR'] = ta.ATR(data['High'], data['Low'], data['Close'].shift(), timeperiod=periodo)
    data.loc[:, 'ATR'] = data['TrueRange'].rolling(20).mean()
    
    for i in range(1, len(data)):

        SATRc1 = abs(data['Close'].iloc[i] - (data['ATR'].iloc[i] * desvio))
        SATRv1 = abs(data['Close'].iloc[i] + (data['ATR'].iloc[i] * desvio))

        if data['Close'].iloc[i] >= satrc[-1]:
            if SATRc1 > satrc[-1]:
                satrc.append(SATRc1)
            else:
                satrc.append(satrc[-1])
        else:
            satrc.append(SATRc1)

        if data['Close'].iloc[i] < satrv[-1]:
            if SATRv1 < satrv[-1]:
                satrv.append(SATRv1)
            else:
                satrv.append(satrv[-1])
        else:
            satrv.append(SATRv1)


    return satrc, satrv

In [3]:
historico = '2y'
intervalo = '1d'
periodo = 20
desvio = 2.75

In [77]:
# Baixar os dados do índice Ibovespa
df = yf.download('^BVSP', period=historico, interval=intervalo)

[*********************100%***********************]  1 of 1 completed


In [119]:
# Criando umas cópia do df
data = df.copy()

# Criar uma nova coluna 'Close Prev' que contenha o valor de fechamento do dia anterior
data.loc[:, 'Close Prev'] = data['Close'].shift(1)

# Calcular o True Range e adicioná-lo ao DataFrame
data.loc[:, 'TrueRange'] = ta.TRANGE(data['High'], data['Low'], data['Close'])


# Calcular o stop atr de compra e venda
"""
satrc, satrv = calculate_satrc_satrv(data, periodo, desvio)

data.loc[:, 'SATRc'] = satrc
data.loc[:, 'SATRv'] = satrv
"""
data.loc[:, 'ATR'] = data['TrueRange'].rolling(20).mean()

data = calcula_stop_atr(data, data['Close'], data['ATR'], desvio, periodo)

# Calcular a média móvel simples do preço de fechamento
data.loc[:, 'SMA'] = data['Close'].rolling(window=periodo).mean()

data.dropna(inplace=True)

In [109]:
"""aux = data.loc['2023-01-01':'2023-03-31'].copy()

aux.loc[:,'ATR_aux'] = abs(aux['Close'] - (aux['ATR'] * desvio))
aux.loc[:, 'Stop ATR_aux'] = np.NAN

for i in range(len(aux)):
    idx = aux.index[i]
    if i < periodo:
        aux.at[idx, 'Stop ATR_aux'] = np.NAN
    elif aux['Close'].iloc[i] >= aux['Stop ATR_aux'].iloc[i-1]:
        if aux['ATR_aux'].iloc[i] > aux['Stop ATR_aux'].iloc[i-1]:
            aux.at[idx, 'Stop ATR_aux'] = aux['ATR_aux'].iloc[i]
        else:
            aux.at[idx, 'Stop ATR_aux'] = aux['Stop ATR_aux'].iloc[i-1]
    else:
        aux.at[idx, 'Stop ATR_aux'] = aux['ATR_aux'].iloc[i]

aux.loc[:, 'Stop ATR C'] = aux['Stop ATR_aux'].shift(1)
aux = aux.drop(['Stop ATR_aux', 'ATR_aux'], axis=1)

aux.loc[:,'ATR_aux'] = abs(aux['Close'] + (aux['ATR'] * desvio))
aux.loc[:, 'Stop ATR_aux'] = np.NAN

for i in range(len(aux)):
    idx = aux.index[i]
    if i < periodo:
        aux.at[idx, 'Stop ATR_aux'] = np.NAN
    elif aux['Close'].iloc[i] < aux['Stop ATR_aux'].iloc[i-1]:
        if aux['ATR_aux'].iloc[i] < aux['Stop ATR_aux'].iloc[i-1]:
            aux.at[idx, 'Stop ATR_aux'] = aux['ATR_aux'].iloc[i]
        else:
            aux.at[idx, 'Stop ATR_aux'] = aux['Stop ATR_aux'].iloc[i-1]
    else:
        aux.at[idx, 'Stop ATR_aux'] = aux['ATR_aux'].iloc[i]

aux.loc[:, 'Stop ATR V'] = aux['Stop ATR_aux'].shift(1)
aux = aux.drop(['Stop ATR_aux', 'ATR_aux'], axis=1)"""

In [121]:
# Plotar os resultados
fig = go.Figure()

# Adicionar gráfico de candlesticks
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close'],
                             name='Candlesticks',
                             increasing_line_width=2,
                             decreasing_line_width=2))

fig.add_trace(go.Scatter(x=data.index[1:], y=data['Stop ATR C'], name='Stop ATR C'))
fig.add_trace(go.Scatter(x=data.index[1:], y=data['Stop ATR V'], name='Stop ATR V'))
#fig.add_trace(go.Scatter(x=data.index, y=data['SMA'], name='SMA', line=dict(color='black', dash='dash')))

fig.update_layout(title='Indicador Stop ATR',
                  xaxis_title='Data',
                  yaxis_title='Valores do Indicador')

fig.show()